In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

data_df = mss_df.loc[(mss_df['UniqueID'].str.contains('WAR')) |
                     (mss_df['UniqueID'].str.contains('WRR')) |
                     (mss_df['UniqueID'].str.contains('BJT'))].copy()

data_df['Tract'] = data_df['Tract'].astype(str).apply(lambda x: x.split('.')[0])
data_df['CMP'] = data_df['CMP'].astype(str).apply(lambda x: x.split('.')[0])
data_df['Stand'] = data_df['Stand'].astype(str).apply(lambda x: x.split('.')[0])
data_df['TCS'] = data_df['Tract'] + "_" + data_df['CMP'] + "_" + data_df['Stand']

data_df['Reporting Period'] = data_df['Reporting Period'].fillna(0)
data_df.loc[data_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = data_df.loc[pd.to_numeric(data_df['Reporting Period'], errors='coerce').isnull() & data_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
data_df['trial'] = data_df.index.map(s)
data_df.loc[pd.to_numeric(data_df['Reporting Period'], errors='coerce').isnull() & data_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = data_df.loc[pd.to_numeric(data_df['Reporting Period'], errors='coerce').isnull() & data_df['Reporting Period'].str.contains("RP"), 'trial']
data_df['Reporting Period'] = pd.to_numeric(data_df['Reporting Period'], downcast='integer')


## Creaet Client Table

In [3]:
# Filter out the Client Provided data and make sure the fields are similar
client_df = data_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Species',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)



# Normalize some of the fields in the client intake table

## Normalize Species
species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['Species_UID'] = species_df.index
species_df['Species'] = species_df['Species'].str.lower()

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Species'] = client_df['Species'].apply(lambda x: species_map(x))


## Normalize origin
origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index
origin_df['Origin'] = origin_df['Origin'].str.lower()

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))


## Normalize tract
client_df['tract_l'] = client_df['Tract'].apply(lambda x: x.lower())
tracts = client_df['tract_l'].unique().tolist()
tract_dict = {'tract': tracts}
tract_df = pd.DataFrame(tract_dict)
tract_df['tract_uid'] = tract_df.index
new_tract_dict = dict(zip(tract_df['tract'], tract_df['tract_uid']))
client_df['Tract'] = client_df['tract_l'].map(new_tract_dict)

## Normalize county
client_df['County'] = client_df['County'].astype(str)
client_df['county_l'] = client_df['County'].apply(lambda x: x.lower())
countys = client_df['county_l'].unique().tolist()
county_dict = {'county': countys}
county_df = pd.DataFrame(county_dict)
county_df['county_uid'] = county_df.index
new_county_dict = dict(zip(county_df['county'], county_df['county_uid']))
client_df['County'] = client_df['county_l'].map(new_county_dict)

## Normalize state
client_df['State'] = client_df['State'].astype(str)
client_df['state_l'] = client_df['State'].apply(lambda x: x.lower())
states = client_df['state_l'].unique().tolist()
state_dict = {'state': states}
state_df = pd.DataFrame(state_dict)
state_df['state_uid'] = state_df.index
new_state_dict = dict(zip(state_df['state'], state_df['state_uid']))
client_df['State'] = client_df['state_l'].map(new_state_dict)

# Drop the lowercase columns
client_df.drop(columns=['tract_l', 'county_l', 'state_l'], inplace=True)


## Create GA Intake Table

In [4]:
ga_intake_df = data_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # This will be the conservative total between client and GA numbers
                       'Notes'
                       ]].copy()

# Lowercase the ga_intake_df column names
ga_intake_df.columns = [x.lower().replace(' ', '_') for x in ga_intake_df.columns]
ga_intake_df.rename(columns={'uniqueid': 'ga_uid'},inplace=True)
ga_intake_df = ga_intake_df[['ga_uid', 'intakeid', 'folderid', 'ga_acres', 'ga_gt', 'total_gt', 'notes']]


## Create Status Table

In [5]:
status_df = data_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()
status_df.rename(columns={'UniqueID': 'status_uid'}, inplace=True)

## Create TicketID -> UniqueID lookup table

In [6]:
ticketid_lookup_df = data_df[['UniqueID', 'TicketID']].copy() # the uniqueid will the the unique id for this table


In [7]:
ticketid_list = data_df['TicketID'].unique().tolist()
ticketid_dict = {'ticketid': ticketid_list}
ticketid_df = pd.DataFrame(ticketid_dict)
ticketid_df['ticketid_uid'] = ticketid_df.index
new_ticketid_dict = dict(zip(ticketid_df['ticketid'], ticketid_df['ticketid_uid']))
ticketid_lookup_df['ticketid'] = ticketid_lookup_df['TicketID'].map(new_ticketid_dict)

ticketid_lookup_df.rename(columns={'ticketid_uid': 'ticket_id',
                                    'UniqueID': 'ticket_lookup_uid'}, inplace=True)
ticketid_lookup_df.drop('TicketID', axis=1, inplace=True)

In [8]:
ticketid_lookup_df

,ticket_lookup_uid,ticketid
10,WAR241025TT01,0
11,WAR241025HD17,1
12,WAR241025HD16,2
13,WAR241025HD15,3
14,WAR241025HD14,3
...,...,...
743,BJT171024TT02,454
744,BJT171024TT01,455
745,BJT171024TH07,456
746,BJT010101TH02,457


## Create the TCS lookup


In [9]:
tcs_df = data_df.groupby('TicketID').agg({'Tract': 'first',
                                               'CMP': 'first',
                                               'Stand': ', '.join}).reset_index()

def split_stands(x):
    bugger_splot = list(set(x.split(', ')))
    bugger_splot.sort()
    return ", ".join(bugger_splot)
    
tcs_df['stands'] = tcs_df['Stand'].apply(split_stands)
tcs_df['comp'] = tcs_df['CMP'].apply(lambda x: x if x!="NP" else "")
tcs_df['TCS'] = tcs_df['Tract'] + "_" + tcs_df['comp'] + "_" + tcs_df['stands']
tcs_df['TCS'] = tcs_df['TCS'].apply(lambda x: x.replace("__", "_"))
tcs_df['tcs_uid'] = tcs_df.index
tcs_df = tcs_df[['tcs_uid', 'TicketID', 'TCS']].copy()

TicketID table should have a link to this

In [10]:
tcs_mapper = dict(zip(tcs_df['TicketID'], tcs_df['tcs_uid']))
ticketid_df['tcs'] = ticketid_df['ticketid'].map(tcs_mapper)

In [11]:
ticketid_df[ticketid_df['tcs'].isna()]

,ticketid,ticketid_uid,tcs


## Place the above tables into the Database
- [X] client_df
- [X] ga_intake_df
- [X] origin_df
- [X] county_df
- [X] species_df
- [X] state_df
- [X] status_df
- [X] tcs_df
- [X] ticketid_df
- [X] ticketid_lookup_df
- [X] tract_df

### Lowercase all of the column names

In [12]:
# Client table
## lowercase client_df field names
client_df.columns = [x.lower() for x in client_df.columns]
client_df.columns = [x.replace(' ', '_') for x in client_df.columns]
client_df['client_uid'] = client_df['uniqueid']
client_df.drop(columns=['uniqueid'],inplace=True)
client_df = client_df[['client_uid', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'species', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',
                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]

# ga_intake
ga_intake_df.columns = [x.lower().replace(' ', '_') for x in ga_intake_df.columns]
ga_intake_df = ga_intake_df[['ga_uid', 'intakeid', 'folderid', 'ga_acres', 'ga_gt', 'total_gt']]

# origin_df
origin_df.rename(columns={'OriginUID': 'origin_uid',
                          'Origin': 'origin'}, inplace=True)

# county_df
county_df.columns = [x.lower() for x in county_df.columns]

# species_df
species_df.columns = [x.lower() for x in species_df.columns]

# state_df
state_df.columns = [x.lower() for x in state_df.columns]

# status_df
status_df.columns = [x.lower() for x in status_df.columns]

# tcs_df
tcs_df.columns = [x.lower() for x in tcs_df.columns]

# ticketid_df
ticketid_df.columns = [x.lower() for x in ticketid_df.columns]

# ticketid_lookup_df
ticketid_lookup_df.columns = [x.lower() for x in ticketid_lookup_df.columns]

# tract_df
tract_df.columns = [x.lower() for x in tract_df.columns]



In [13]:
# define the db
import sqlite3

conn = sqlite3.connect(r'C:\Users\bbrown\Documents\python_scripts\db-build-trial\db_trial_v1\ga_harvest.db')
c = conn.cursor()

In [14]:
### Update the DB

client_df.to_sql('client_intake', conn, if_exists='replace',index=False, dtype={'client_uid': 'TEXT PRIMARY KEY'})
ga_intake_df.to_sql('ga_intake', conn, if_exists='replace', index=False, dtype={'ga_uid': 'TEXT PRIMARY KEY'})
origin_df.to_sql('origin', conn, if_exists='replace', index=False, dtype={'origin_uid': 'INTEGER PRIMARY KEY'})
county_df.to_sql('county', conn, if_exists='replace', index=False, dtype={'county_uid': 'INTEGER PRIMARY KEY'})
species_df.to_sql('species', conn, if_exists='replace', index=False, dtype={'species_uid': 'INTEGER PRIMARY KEY'})
state_df.to_sql('state', conn, if_exists='replace', index=False, dtype={'state_uid': 'INTEGER PRIMARY KEY'})
status_df.to_sql('status', conn, if_exists='replace', index=False, dtype={'status_uid': 'TEXT PRIMARY KEY'})
tcs_df.to_sql('tcs', conn, if_exists='replace', index=False, dtype={'tcs_uid': 'INTEGER PRIMARY KEY'})
ticketid_df.to_sql('ticketid', conn, if_exists='replace', index=False, dtype={'ticketid_uid': 'INTEGER PRIMARY KEY'})
ticketid_lookup_df.to_sql('ticketid_lookup_activityid', conn, if_exists='replace', index=False, dtype={'ticketid_lookup_uid': 'TEXT PRIMARY KEY'})
tract_df.to_sql('tract', conn, if_exists='replace', index=False, dtype={'tract_uid':'INTEGER PRIMARY KEY'})


122

# Mill Ticket sections

In [15]:
projs = ['WAR', 'WRR', 'BJT']

In [16]:
mill_loc = r"P:\Mill Tickets"
mega_df = pd.DataFrame()

for proj in projs:
    file_pattern = f"{proj}_RP*_Compiled.xlsm"
    big_df = pd.DataFrame()

    mill_name = []
    for root, dirs, files in os.walk(mill_loc):
        for file in files:
            if f"{proj}_RP" in file and file.endswith("Compiled.xlsm"):
                if "$" in file:
                    pass
                else:
                    print(root)
                    print(file)
                    df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                    big_df = pd.concat([big_df, df])
    mega_df = pd.concat([mega_df, big_df])
                    
'''
    big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
    big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
    big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
    big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

    def rep_fix(x):
        if "RP" in str(x):
            return x[-1]
        else:
            return x
        
    big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
    big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

    big_df['Date'] = pd.to_datetime(big_df['Date'])

    harvest_df = big_df[['Date',
                    'Ticket',
                    'Mill',
                    'Product',
                    'Tons (U.S)',
                    'Cull Tons',
                    'Accepted Tons',
                    'Source',
                    'Notes',
                    'Reporting Period',
                    'TicketID',
                    'Week']].copy()

    harvest_df = harvest_df.drop_duplicates()
    '''

P:\Mill Tickets\ARCHIVE\WAR_RP6
WAR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP7
WAR_RP7_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP8
WAR_RP8_Compiled.xlsm
P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP2
WAR_RP2_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP3
WAR_RP3_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP4
WAR_RP4_Compiled.xlsm


...

c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\WAR_RP9
WAR_RP9_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP5
WRR_RP5_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP6
WRR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP7
WRR_RP7_Compiled.xlsm
P:\Mill Tickets\WRR_RP8
WRR_RP8_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\BJT_RP5
BJT_RP5_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\BJT_RP6
BJT_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\BJT_RP7
BJT_RP7_Compiled.xlsm


C:\Users\bbrown\AppData\Local\Temp\ipykernel_4352\2111848845.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat([big_df, df])


P:\Mill Tickets\BJT_RP8
BJT_RP8_Compiled.xlsm


'\n    big_df.loc[big_df[\'Tons (U.S)\'].isnull(), \'Tons (U.S)\'] = big_df.loc[big_df[\'Tons (U.S)\'].isnull(), \'TonsUS\']\n    big_df[\'Cull Tons\'].fillna(big_df[\'CullTons\'], inplace=True)\n    big_df[\'Accepted Tons\'].fillna(big_df[\'AcceptedTons\'], inplace=True)\n    big_df[\'Reporting Period\'].fillna(big_df[\'ReportingPeriod\'],inplace=True)\n\n    def rep_fix(x):\n        if "RP" in str(x):\n            return x[-1]\n        else:\n            return x\n        \n    big_df[\'Reporting Period\'] = big_df[\'Reporting Period\'].apply(lambda x: rep_fix(x))\n    big_df[\'Reporting Period\'] = pd.to_numeric(big_df[\'Reporting Period\'])\n\n    big_df[\'Date\'] = pd.to_datetime(big_df[\'Date\'])\n\n    harvest_df = big_df[[\'Date\',\n                    \'Ticket\',\n                    \'Mill\',\n                    \'Product\',\n                    \'Tons (U.S)\',\n                    \'Cull Tons\',\n                    \'Accepted Tons\',\n                    \'Source\',\n     

In [17]:
mega_df.loc[mega_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = mega_df.loc[mega_df['Tons (U.S)'].isnull(), 'TonsUS']
mega_df['Cull Tons'].fillna(mega_df['CullTons'], inplace=True)
mega_df['Accepted Tons'].fillna(mega_df['AcceptedTons'], inplace=True)
mega_df['Reporting Period'].fillna(mega_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
mega_df['Reporting Period'] = mega_df['Reporting Period'].apply(lambda x: rep_fix(x))
mega_df['Reporting Period'] = pd.to_numeric(mega_df['Reporting Period'])

mega_df['Date'] = pd.to_datetime(mega_df['Date'])

harvest_df = mega_df[['Date',
                'Ticket',
                'Mill',
                'Product',
                'Tons (U.S)',
                'Cull Tons',
                'Accepted Tons',
                'Source',
                'Notes',
                'Reporting Period',
                'TicketID',
                'Week']].copy()

harvest_df = harvest_df.drop_duplicates()
harvest_df.loc[harvest_df['TicketID'] == 'WAR18051XXXX', 'TicketID'] = 'WAR180518XXXX' # Found some mislabeled tickets
harvest_df.reset_index(inplace=True)

In [18]:
mill_list = harvest_df['Mill'].unique().tolist()
mill_dict = {}
mill_dict['mill'] = mill_list
mill_df = pd.DataFrame(mill_dict)
mill_df['mill_uid'] = mill_df.index
mill_mapper = dict(zip(mill_df['mill'], mill_df['mill_uid']))
harvest_df['mill'] = harvest_df['Mill'].map(mill_mapper)
harvest_df.drop(columns=['Mill'], inplace=True)

In [19]:
harvest_df['Week'] = harvest_df['Week'].astype(str)
harvest_df.loc[harvest_df['Week'] == 'Week3b', 'Week'] = "3"
def weeker(week):
    if "Week" in week:
        x = week.split("k")[1]
        x = int(x)
        return x
    elif "nan" in week:
        return 0
    else:
        return int(week)
    
harvest_df['Week_trial'] = harvest_df['Week'].apply(lambda x: weeker(x))

# I am not sure if I want to normalize the week or just leave it as the integer it is...
'''week_list = harvest_df['Week_trial'].unique().tolist()
week_dict = {}
week_dict['week'] = week_list
week_df = pd.DataFrame(week_dict)
week_df['week_uid'] = week_df.index
week_mapper = dict(zip(week_df['week'], week_df['week_uid']))
harvest_df['week'] = harvest_df['Week_trial'].map(week_mapper)'''

harvest_df['week'] = harvest_df['Week_trial']

In [20]:
product_list = harvest_df['Product'].unique().tolist()
product_dict = {}
product_dict['product'] = product_list
product_df = pd.DataFrame(product_dict)
product_df['product_uid'] = product_df.index
product_mapper = dict(zip(product_df['product'], product_df['product_uid']))
harvest_df['product'] = harvest_df['Product'].map(product_mapper)

In [21]:
ticket_id_mapper = dict(zip(ticketid_df['ticketid'], ticketid_df['ticketid_uid']))
harvest_df['ticketid'] = harvest_df['TicketID'].map(ticket_id_mapper)
harvest_df['ticketid'] = harvest_df['ticketid'].astype(int)
harvest_df.drop(columns=['Product', 'TicketID', 'Week', 'Week_trial', 'index'],inplace=True)
harvest_df.rename(columns={'Tons (U.S)': 'total_tons'})
harvest_df['harvest_uid'] = harvest_df.index

In [22]:
harvest_df

,Date,Ticket,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,mill,week,product,ticketid,harvest_uid
0,2021-08-02,160048,26.31,0.0,26.31,M-82_Week 1.pdf,NaN,6,0,1,0,375,0
1,2021-08-03,160157,25.79,0.0,25.79,M-82_Week 1.pdf,NaN,6,0,1,0,375,1
2,2021-08-03,160120,25.32,0.0,25.32,M-82_Week 1.pdf,,6,0,1,0,375,2
3,2021-08-03,160155,25.31,0.0,25.31,M-82_Week 1.pdf,,6,0,1,0,375,3
4,2021-08-03,160082,27.17,0.0,27.17,M-82_Week 1.pdf,,6,0,1,0,375,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48801,2024-07-15,21640715,30.05,0.0,30.05,2024BJT39wk2.pdf,NaN,8,40,4,0,432,48801
48802,2024-07-24,21644553,32.88,0.0,32.88,2024BJT39wk2.pdf,NaN,8,40,4,0,432,48802
48803,2024-07-24,21644635,26.47,0.0,26.47,2024BJT39wk2.pdf,NaN,8,40,4,0,432,48803
48804,2024-07-22,21643437,32.48,0.0,32.48,2024BJT39wk2.pdf,NaN,8,40,4,0,432,48804


In [23]:
# Send harvest data to db

harvest_df.to_sql('harvest_vol', conn, if_exists='replace', index=False, dtype={'harvest_uid':'INTEGER PRIMARY KEY'})
product_df.to_sql('product', conn, if_exists='replace', index=False, dtype={'product_uid': 'INTEGER PRIMARY KEY'})
mill_df.to_sql('mill', conn, if_exists='replace', index=False, dtype={'mill_uid': 'INTEGER PRIMARY KEY'})

121

## Time to make the Expected Tons / RP table


My plan:
- Get the previous RP expected simply from MSS 
- Current RP pull from files I made for Rob



In [24]:
# The following should get me up to RP8 for WAR
war_expected_inprog = data_df[(~data_df['Reporting Period'].isna()) & (data_df['UniqueID'].str.contains("WAR"))].copy()
war_expected_inprog = war_expected_inprog[war_expected_inprog['Reporting Period'] != 9]
war_expected_inprog = war_expected_inprog[['TicketID', 'Reporting Period', 'Current RP Expected GT']]
war_expected_inprog = war_expected_inprog[war_expected_inprog['Current RP Expected GT'] != ""]
war_expected_inprog = war_expected_inprog.groupby(['TicketID', 'Reporting Period'])['Current RP Expected GT'].sum().reset_index()

In [25]:
war_expected_inprog.head()

,TicketID,Reporting Period,Current RP Expected GT
0,WAR211101CC01,6,0.0
1,WAR220728XXXX,8,6177.0
2,WAR220729TH01,8,17604.0
3,WAR220801CC01,8,3268.0
4,WAR220801CC03,8,10324.0


In [73]:
war_expected_rp9 = pd.read_excel(r"./WAR_RP9_InProcess/Excel_sheets/rp9_investigate_matchesRob.xlsx", sheet_name='Sheet2')
war_expected_rp9.rename(columns=({'RP9 Expected': 'Current RP Expected GT'}), inplace=True)

In [74]:
war_expected_rp9['Reporting Period'] = 9
war_expected_rp9, war_expected_rp9['Current RP Expected GT'].sum()

(         TicketID  Current RP Expected GT  Reporting Period
 0   WAR230908XXXX                 9019.00                 9
 1   WAR231107XXXX                12289.71                 9
 2   WAR240521CCXX                 9814.68                 9
 3   WAR240521TN01                 2918.65                 9
 4   WAR240521TT01                 6756.53                 9
 ..            ...                     ...               ...
 68  WAR241025HD04                 5186.26                 9
 69  WAR241025HD03                 2700.00                 9
 70  WAR241025HD02                 8592.47                 9
 71  WAR241025HD01                 1200.00                 9
 72  WAR230318TXXX                    0.00                 9
 
 [73 rows x 3 columns],
 424209.6018388594)

In [75]:
# Join the two together - though it will not account for any carryover pre- RP8 ( i am pretty sure I brute forced the RP8 to be correct for reporting purposes)
war_expected = pd.concat([war_expected_inprog, war_expected_rp9])


In [76]:
# Let's do WRR
wrr_expected_inprog = data_df[(~data_df['Reporting Period'].isna()) & (data_df['UniqueID'].str.contains("WRR"))].copy()
wrr_expected_inprog = wrr_expected_inprog[wrr_expected_inprog['Reporting Period'] != 8]
wrr_expected_inprog = wrr_expected_inprog[['TicketID', 'Reporting Period', 'Current RP Expected GT']]
wrr_expected_inprog = wrr_expected_inprog[wrr_expected_inprog['Current RP Expected GT'] != ""]
wrr_expected_inprog = wrr_expected_inprog.groupby(['TicketID', 'Reporting Period'])['Current RP Expected GT'].sum().reset_index()

# Current rp
wrr_current_rp = pd.read_excel(r"C:\Users\bbrown\Documents\python_scripts\db-build-trial\WRR_RP8_InProcess\WRR_to_db_241202.xlsx")

In [77]:
wrr_expected = pd.concat([wrr_expected_inprog, wrr_current_rp])

In [78]:
wrr_expected.loc[wrr_expected['Reporting Period'] == 8, 'Current RP Expected GT'].sum()

84259.66040039062

In [79]:
# Time for BJT
# previous rp expected
bjt_expected_inprog = data_df[(~data_df['Reporting Period'].isna()) & (data_df['UniqueID'].str.contains("BJT"))].copy()
bjt_expected_inprog = bjt_expected_inprog[bjt_expected_inprog['Reporting Period'] != 8]
bjt_expected_inprog = bjt_expected_inprog[['TicketID', 'Reporting Period', 'Current RP Expected GT']]
bjt_expected_inprog = bjt_expected_inprog[bjt_expected_inprog['Current RP Expected GT'] != '']
bjt_expected_inprog = bjt_expected_inprog.groupby(['TicketID', 'Reporting Period'])['Current RP Expected GT'].sum().reset_index()

# current rp expected
bjt_current_rp = pd.read_excel(r"C:\Users\bbrown\Documents\python_scripts\db-build-trial\BJT_RP8_InProcess\BJT_RP8_requestvol_241202.xlsx")

In [83]:
bjt_expected_df = pd.concat([bjt_expected_inprog, bjt_current_rp])

In [84]:
# Group all of the expected into a single table

total_expected = pd.concat([war_expected, wrr_expected, bjt_expected_df])

In [85]:
total_expected.reset_index(inplace=True)
total_expected['gt_expected_uid'] = total_expected.index
# Normalize the ticketid
total_expected['ticketid'] = total_expected['TicketID'].map(ticket_id_mapper)
total_expected.to_excel("expected_lookup.xlsx")
total_expected['ticketid'] = total_expected['ticketid'].astype(int)
total_expected = total_expected.rename(columns={'Reporting Period': 'reporting_period', 'Current RP Expected GT': 'expected_gt'})
total_expected = total_expected[['gt_expected_uid', 'ticketid', 'reporting_period', 'expected_gt']]





total_expected.to_sql('gt_expected', conn, index=False, if_exists='replace', dtype={'gt_expected_uid': 'INTEGER PRIMARY KEY'})

282